In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import string
import os
%pip install openai
os.chdir('/content/drive/MyDrive/Sanjiv/IMP-OIC-Windowing')
from STAR_eval.gpt_ask import run_gpt
main_ds = pd.read_json('STAR_eval/STAR_val.json')
main = main_ds.loc[:, ['question_id','question','video_id','start','end','answer', 'choices']]
os.chdir('/content/drive/MyDrive/Sanjiv/IMP-OIC-Windowing/datatables')


In [ ]:
csv_files = [f for f in os.listdir() if f.endswith('.csv')]

In [ ]:
dfs = []

for csv in csv_files:
    df = pd.read_csv(csv)
    dfs.append(df)

In [ ]:
final_df = pd.concat(dfs, ignore_index=True)
final_df.drop_duplicates()
final_df.reindex()
final_df.head()

videos = final_df['question_id'].unique()
print(videos.size)

videos = main['video_id'].unique()
print(videos.size)

main.info

In [ ]:
start = main['start']
end = main['end']

duration = end - start

print(duration.mean())

In [ ]:
CLASSES = ['airplane', 'animal', 'arm', 'bag', 'banana', 'basket', 'beach', 'bear', 'bed', 'bench', 'bike',
                'bird', 'board', 'boat', 'book', 'boot', 'bottle', 'bowl', 'box', 'boy', 'branch', 'building',
                'bus', 'cabinet', 'cap', 'car', 'cat', 'chair', 'child', 'clock', 'coat', 'counter', 'cow', 'cup',
                'curtain', 'desk', 'dog', 'door', 'drawer', 'ear', 'elephant', 'engine', 'eye', 'face', 'fence',
                'finger', 'flag', 'flower', 'food', 'fork', 'fruit', 'giraffe', 'girl', 'glass', 'glove', 'guy',
                'hair', 'hand', 'handle', 'hat', 'head', 'helmet', 'hill', 'horse', 'house', 'jacket', 'jean',
                'kid', 'kite', 'lady', 'lamp', 'laptop', 'leaf', 'leg', 'letter', 'light', 'logo', 'man', 'men',
                'motorcycle', 'mountain', 'mouth', 'neck', 'nose', 'number', 'orange', 'pant', 'paper', 'paw',
                'people', 'person', 'phone', 'pillow', 'pizza', 'plane', 'plant', 'plate', 'player', 'pole', 'post',
                'pot', 'racket', 'railing', 'rock', 'roof', 'room', 'screen', 'seat', 'sheep', 'shelf', 'shirt',
                'shoe', 'short', 'sidewalk', 'sign', 'sink', 'skateboard', 'ski', 'skier', 'sneaker', 'snow',
                'sock', 'stand', 'street', 'surfboard', 'table', 'tail', 'tie', 'tile', 'tire', 'toilet', 'towel',
                'tower', 'track', 'train', 'tree', 'truck', 'trunk', 'umbrella', 'vase', 'vegetable', 'vehicle',
                'wave', 'wheel', 'window', 'windshield', 'wing', 'wire', 'woman', 'zebra']

REL_CLASSES = ['above', 'across', 'against', 'along', 'and', 'at', 'attached to', 'behind',
                'belonging to', 'between', 'carrying', 'covered in', 'covering', 'eating', 'flying in', 'for',
                'from', 'growing on', 'hanging from', 'has', 'holding', 'in', 'in front of', 'laying on',
                'looking at', 'lying on', 'made of', 'mounted on', 'near', 'of', 'on', 'on back of', 'over',
                'painted on', 'parked on', 'part of', 'playing', 'riding', 'says', 'sitting on', 'standing on',
                'to', 'under', 'using', 'walking in', 'walking on', 'watching', 'wearing', 'wears', 'with']

words = set(CLASSES+REL_CLASSES)

In [ ]:
import string
index_list = []
v_id =
for q in QA.loc[:,'question_id'].values:
    questions = QA.query("question_id=='"+q+"'")["question"]
    answer = QA.query("question_id=='"+q+"'")["answer"].values[0].lower().strip('the').translate(str.maketrans('', '', string.punctuation))
    count = 0
    question = ''
    for que in questions:
        question = que.lower().translate(str.maketrans('', '', string.punctuation))

    for i in list(answer.strip().split(' ')) + list(question.strip().split(' ')):
      if i not in  ['a', 'is','by', 'was', 'which', 'man', 'did', 'do', 'woman', 'boy', 'girl', 'person','people','scene','frame','in','to','the', 'of', 'on','with', 'from', 'at', 'and']:
          if i in words:
            index_list.append(QA[QA['question_id']==q].index[0])

print(index_list)
#QA.drop(index_list, inplace = True)
QA = QA[QA.index.isin(index_list)]
QA.info()


In [ ]:
import string
import time

ds = QA
q_ids = ds['question_id'].unique()
v_ids = ds['video_id'].unique()
ds = ds.drop(columns =['Match'])
for q in q_ids:
  que = ds.query("question_id == '"+q+"'")
  answer = que['answer'].values[0].lower().strip('the ').translate(str.maketrans('', '', string.punctuation))
  OIC_answer = que['OIC_answer'].values[0]
  filter_words = ['not', 'no', 'none']
  if 'not' in OIC_answer or 'no' in OIC_answer or 'none' in OIC_answer:
    ds.loc[ds['question_id'] == q, 'Match'] = 'Wrong'
  elif answer in OIC_answer:
      ds.loc[ds['question_id'] == q, 'Match'] = 'Correct'
  else:
    ds.loc[ds['question_id'] == q, 'Match'] = 'Wrong'

ds.head()
ds.name = 'all data'
print(v_ids.size)

In [ ]:

for i in ds['question_id']:
  que = ds.query("question_id == '"+i+"'")
  questions = que["question"].values[0]
  answer = que["answer"].values[0].lower().strip('the').translate(str.maketrans('', '', string.punctuation))
  choices = dict()
  choice_string = ''
  options = main.query("question_id == '"+i+"'")["choices"].values[0]
  for choice in options:
        choices.update({choice['choice_id']:choice['choice'].lower().strip('the').translate(str.maketrans('', '', string.punctuation))})
        choice_string += ' ('+str(choice['choice_id']+1)+')'+str(choice['choice'].lower().strip('the').translate(str.maketrans('', '', string.punctuation)))

  formated_question = questions+' Guess the most likely answer among these four options: '+choice_string+' Respond only with a single number between 1 and 4. Do not produce any other output.'
  print('Question : {}'.format(formated_question))
  print('Expected Answer: {}'.format(answer))
  context = QA.query("question_id=='"+i+"'")["OIC_context"].values[0]
  print('Prompt for LLM: {}'.format(context))
  oic_answer = run_gpt(context, formated_question)
  print('OIC answer: choice- {}, vlaue- {}'.format(oic_answer, choices[int(oic_answer)-1]))

  if choices[int(oic_answer)-1] == answer:
    ds.loc[ds['question_id'] == i, 'new_Match'] = 'Correct'
    print('correct')
  else:
    print('wrong')
    ds.loc[ds['question_id'] == i, 'new_Match'] = 'Wrong'
  print('------------------------------------')





In [ ]:
interaction_ds = ds.loc[ds['question_id'].str.contains('Interaction')]
interaction_ds.name='Interaction'
interaction_ds.head()


In [ ]:
sequence_ds = ds.loc[ds['question_id'].str.contains('Sequence')]
sequence_ds.name = 'Sequence'
sequence_ds.head()

In [ ]:
prediction_ds = ds.loc[ds['question_id'].str.contains('Prediction')]
prediction_ds.name='Prediction'
prediction_ds.head()

In [ ]:
feasibility_ds = ds.loc[ds['question_id'].str.contains('Feasibility')]
feasibility_ds.name= 'Feasibility'
feasibility_ds.head()

In [ ]:
for i in [ds, interaction_ds,sequence_ds,prediction_ds, feasibility_ds]:
  count = (i['new_Match'].value_counts()['Correct'])
  length = len(i)
  print('length of {} is: {}'.format(i.name, length))
  print('Correct match percentage for {} is: {}'.format(i.name, count/length))

In [ ]:
context = 'In frame 0: woman_6109 wearing shirt_3a73.woman_6109 has hair_cfe9.woman_6109 wearing pant_b4d2.woman_6109 on chair_1702.woman_6109 has hand_3b94.woman_6109 has hand_f64a.shirt_3a73 on woman_6109.hair_cfe9 on woman_6109.pant_b4d2 on woman_6109.head_366f of woman_6109.arm_2c4a of woman_6109.bottle_c5b4 on counter_6d66.sink_af57 on counter_6d66.hand_f64a of woman_6109.In frame 1: woman_6109 wearing shirt_3a73.woman_6109 has hair_cfe9.woman_6109 wearing pant_b4d2.woman_6109 on chair_1702.woman_6109 has hand_f64a.woman_6109 has arm_2c4a.woman_6109 has hand_7c4d.woman_6109 has hand_a7ca.shirt_3a73 on woman_6109.hair_cfe9 on woman_6109.head_366f of woman_6109.bottle_c5b4 on counter_6d66.sink_af57 on counter_6d66.sink_af57 on counter_314c.curtain_35a8 over window_3e58.bottle_13db on counter_6d66.In frame 2: woman_6109 wearing shirt_3a73.woman_6109 has hair_cfe9.woman_6109 wearing pant_b4d2.woman_6109 on chair_1702.woman_6109 has hand_4822.shirt_3a73 on woman_6109.hair_cfe9 on woman_6109.bottle_c5b4 on counter_6d66.bottle_13db on counter_6d66.In frame 3: woman_6109 wearing shirt_3a73.woman_6109 has hair_cfe9.woman_6109 wearing pant_b4d2.woman_6109 on chair_1702.woman_6109 has arm_2c4a.woman_6109 has hand_4822.woman_6109 has hand_2613.shirt_3a73 on woman_6109.pant_b4d2 on woman_6109.bottle_c5b4 on counter_6d66.sink_af57 on counter_6d66.sink_af57 on counter_314c.curtain_35a8 over window_3e58.In frame 4: woman_6109 wearing shirt_3a73.woman_6109 has hair_cfe9.woman_6109 wearing pant_b4d2.woman_6109 on chair_1702.woman_6109 wearing glass_e33c.shirt_3a73 on woman_6109.pant_b4d2 on woman_6109.bottle_c5b4 on counter_6d66.sink_af57 on counter_6d66.sink_af57 on counter_314c.curtain_35a8 over window_3e58.bottle_13db on counter_6d66.glass_e33c on woman_6109.In frame 5: woman_6109 wearing shirt_3a73.woman_6109 has hair_cfe9.woman_6109 wearing pant_b4d2.woman_6109 on chair_1702.woman_6109 wearing glass_e33c.shirt_3a73 on woman_6109.pant_b4d2 on woman_6109.bottle_c5b4 on counter_6d66.sink_af57 on counter_6d66.sink_af57 on counter_314c.curtain_35a8 over window_3e58.bottle_13db on counter_6d66.glass_e33c on woman_6109.In frame 6: woman_6109 wearing shirt_3a73.woman_6109 has hair_cfe9.woman_6109 wearing pant_b4d2.woman_6109 on chair_1702.shirt_3a73 on woman_6109.pant_b4d2 on woman_6109.bottle_c5b4 on counter_6d66.sink_af57 on counter_6d66.sink_af57 on counter_314c.curtain_35a8 on window_3e58.bottle_13db on counter_6d66.bottle_13db on counter_314c.In frame 7: woman_6109 wearing shirt_3a73.woman_6109 has hair_cfe9.woman_6109 wearing pant_b4d2.woman_6109 on chair_1702.shirt_3a73 on woman_6109.pant_b4d2 on woman_6109.bottle_c5b4 on counter_6d66.sink_af57 on counter_6d66.sink_af57 on counter_314c.curtain_35a8 on window_3e58.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.In frame 8: bottle_c5b4 on counter_6d66.sink_af57 on counter_6d66.sink_af57 on counter_314c.curtain_35a8 over window_3e58.bottle_13db on counter_6d66.bottle_13db on counter_314c.bottle_13db near sink_af57.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.girl_eb23 on chair_1702.In frame 9: bottle_c5b4 on counter_6d66.sink_af57 on counter_6d66.bottle_13db on counter_6d66.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.girl_eb23 on chair_1702.girl_eb23 has hand_01f2.In frame 10: bottle_c5b4 on counter_6d66.sink_af57 on counter_6d66.bottle_13db on counter_6d66.bottle_13db on counter_314c.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.girl_eb23 on chair_1702.girl_eb23 has hand_8cde.In frame 11: shirt_3a73 on girl_eb23.bottle_c5b4 on counter_6d66.sink_af57 on counter_6d66.curtain_35a8 on window_d1db.bottle_13db on counter_6d66.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.girl_eb23 on chair_1702.girl_eb23 sitting on chair_29c0.chair_29c0 at table_c710.chair_29c0 at counter_4559.In frame 12: woman_6109 wearing shirt_3a73.woman_6109 has hair_cfe9.woman_6109 wearing pant_b4d2.woman_6109 on chair_1702.woman_6109 has hand_4822.woman_6109 has hat_44a8.shirt_3a73 on woman_6109.pant_b4d2 on woman_6109.bottle_c5b4 on counter_6d66.bottle_13db on counter_6d66.bottle_13db on counter_314c.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.chair_29c0 at table_c710.In frame 13: shirt_3a73 on girl_eb23.hair_cfe9 on girl_eb23.pant_b4d2 on girl_eb23.bottle_c5b4 on counter_314c.curtain_35a8 on window_d1db.bottle_13db on counter_6d66.bottle_13db on counter_314c.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.girl_eb23 on chair_1702.girl_eb23 has hair_4439.In frame 14: shirt_3a73 on girl_eb23.hair_cfe9 on girl_eb23.pant_b4d2 on girl_eb23.bottle_c5b4 on counter_6d66.bottle_13db on counter_6d66.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.girl_eb23 on chair_1702.girl_eb23 has hair_4439.man_f230 wearing hat_7a75.man_f230 wearing shirt_5427.In frame 15: shirt_3a73 on girl_eb23.hair_cfe9 on girl_eb23.pant_b4d2 on girl_eb23.curtain_35a8 on window_d1db.bottle_13db on counter_6d66.girl_eb23 in shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.girl_eb23 sitting on chair_1702.In frame 16: shirt_3a73 on girl_eb23.pant_b4d2 on girl_eb23.bottle_c5b4 on counter_6d66.bottle_13db on counter_6d66.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.man_f230 wearing shirt_5427.In frame 17: shirt_3a73 on girl_eb23.pant_b4d2 on girl_eb23.bottle_c5b4 on counter_6d66.curtain_35a8 on window_d1db.bottle_13db on counter_6d66.bottle_13db on counter_314c.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.girl_eb23 sitting on chair_1702.girl_eb23 holding bottle_13db.chair_29c0 at counter_48c0.In frame 18: shirt_3a73 on girl_eb23.pant_b4d2 on girl_eb23.bottle_13db on counter_6d66.bottle_13db on counter_86ea.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.girl_eb23 sitting on chair_1702.girl_eb23 sitting on chair_29c0.In frame 19: woman_6109 wearing pant_b4d2.shirt_3a73 on girl_eb23.pant_b4d2 on girl_eb23.bottle_c5b4 on counter_6d66.bottle_13db on counter_6d66.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.girl_eb23 sitting on chair_1702.girl_eb23 sitting on chair_29c0.In frame 20: woman_6109 wearing shirt_3a73.woman_6109 wearing pant_b4d2.shirt_3a73 on girl_eb23.pant_b4d2 on girl_eb23.bottle_c5b4 on counter_6d66.curtain_35a8 on window_d1db.bottle_13db on counter_6d66.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.girl_eb23 on chair_1702.girl_eb23 has hand_8cde.girl_eb23 sitting on chair_29c0.In frame 21: woman_6109 has hair_cfe9.woman_6109 sitting on chair_29c0.shirt_3a73 on woman_6109.pant_b4d2 on woman_6109.bottle_13db on counter_6d66.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.girl_eb23 sitting on chair_29c0.girl_eb23 sitting on chair_09d0.In frame 22: woman_6109 wearing shirt_3a73.woman_6109 has hair_cfe9.woman_6109 sitting on chair_1702.woman_6109 sitting on chair_29c0.woman_6109 sitting on chair_ccea.shirt_3a73 on woman_6109.pant_b4d2 on woman_6109.bottle_c5b4 on counter_6d66.curtain_35a8 on window_d1db.bottle_13db on counter_6d66.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.girl_eb23 sitting on chair_29c0.chair_29c0 at table_c710.chair_ccea at table_c710.In frame 23: woman_6109 in shirt_3a73.woman_6109 has hair_cfe9.woman_6109 wearing pant_b4d2.woman_6109 sitting on chair_ccea.shirt_3a73 on woman_6109.pant_b4d2 on woman_6109.bottle_13db on counter_6d66.chair_ccea at table_c710.In frame 24: woman_6109 in shirt_3a73.woman_6109 has hair_cfe9.woman_6109 wearing pant_b4d2.woman_6109 sitting on chair_1702.shirt_3a73 on woman_6109.pant_b4d2 on woman_6109.bottle_13db on counter_6d66.chair_ccea at table_c710.In frame 25: woman_6109 has hair_cfe9.woman_6109 sitting on chair_1702.bottle_13db on counter_6d66.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.chair_ccea at table_c710.boy_8d1a wearing shirt_3a73.boy_8d1a wearing pant_b4d2.boy_8d1a has hair_cfe9.In frame 26: woman_6109 has hair_cfe9.bottle_13db on counter_6d66.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.girl_eb23 has hand_01f2.girl_eb23 sitting on chair_ccea.chair_ccea at table_c710.boy_8d1a wearing shirt_3a73.boy_8d1a wearing pant_b4d2.boy_8d1a has hair_cfe9.In frame 27: bottle_13db on counter_6d66.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.girl_eb23 sitting on chair_ccea.chair_ccea at table_c710.In frame 28: woman_6109 wearing shirt_3a73.woman_6109 has hair_cfe9.woman_6109 wearing pant_b4d2.shirt_3a73 on woman_6109.pant_b4d2 on woman_6109.bottle_c5b4 on counter_6d66.bottle_13db on counter_6d66.chair_ccea at table_c710.boy_8d1a wearing shirt_3a73.boy_8d1a wearing pant_b4d2.boy_8d1a has hair_cfe9.glass_5a0b on table_c710.In frame 29: woman_6109 wearing shirt_3a73.woman_6109 has hair_cfe9.woman_6109 wearing pant_b4d2.woman_6109 sitting on chair_1702.bottle_c5b4 on counter_314c.curtain_35a8 on window_d1db.bottle_13db on counter_314c.chair_ccea at table_c710.boy_8d1a wearing shirt_3a73.boy_8d1a wearing pant_b4d2.boy_8d1a has hair_cfe9.plant_5bab on counter_314c.curtain_f5df on window_3e58.'
answer = run_gpt(context, 'Which object did the person put down after they watched outside the window? Guess the most likely answer among these four options:  (1) shoe. (2) cup/glass/bottle. (3) towel. (4) box. Respond only with a single number between 1 and 4. Do not produce any other output.')
print('OIC answer: {}'.format(answer))